In [ ]:
!pip install spacy==3.0.0
!pip install ml-datasets
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.6/660.6 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 34.7 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.4.2
    Uninstalling typer-0.4.2:
      Successfully uninstalled typer-0.4.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.2
    Uninstalling spacy-3.4.2:
      Successfully uninstalled spacy-3.4.2
ERROR: pip's dependency resolver does not currently t

In [ ]:
import spacy
from tqdm.auto import tqdm
from spacy.tokens import DocBin
from ml_datasets import imdb
train_data, valid_data = imdb()
nlp = spacy.load("en_core_web_md")

84131840it [00:01, 49734062.53it/s]                              


Untaring file...


/usr/local/lib/python3.7/dist-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_md' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  except PermissionError as e:


In [ ]:
train_data[0]

('Finally I got to see the infamous "Ice Age". Apart from maybe not being as dead funny as I\'d hoped for after seeing the brilliant teaser there is not a bad word I can say about it. Sure, it\'s not as glamorous as a Disney production(besides, it is Fox\'s 1st attempt at a full length CG movie) but it\'s got immense heart and on some occasions(like the look in Manfred\'s eyes after we see the sad glimpse of his past) I found myself on the verge of tears. But when they reunited the baby with its father I just couldn\'t hold them in anymore. A movie that has no trouble walking on the thin line of sappy and cliché and manages to bring more than the best out of it; the end result being one of the most touching animated creations I have ever seen. Great funny looking characters that quickly grow on you(and great voice talents as well) and many funny memorable scenes, especially from Scrat\'s behalf make the movie more than enough reason to give it a go. Plus the Dodo scene, which is my per

In [ ]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == 'neg':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [ ]:
train_data[0]

('Finally I got to see the infamous "Ice Age". Apart from maybe not being as dead funny as I\'d hoped for after seeing the brilliant teaser there is not a bad word I can say about it. Sure, it\'s not as glamorous as a Disney production(besides, it is Fox\'s 1st attempt at a full length CG movie) but it\'s got immense heart and on some occasions(like the look in Manfred\'s eyes after we see the sad glimpse of his past) I found myself on the verge of tears. But when they reunited the baby with its father I just couldn\'t hold them in anymore. A movie that has no trouble walking on the thin line of sappy and cliché and manages to bring more than the best out of it; the end result being one of the most touching animated creations I have ever seen. Great funny looking characters that quickly grow on you(and great voice talents as well) and many funny memorable scenes, especially from Scrat\'s behalf make the movie more than enough reason to give it a go. Plus the Dodo scene, which is my per

In [ ]:
num_texts = 1000
train_docs = make_docs(train_data[:num_texts])

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./output

ℹ Using CPU

=========================== Initializing pipeline ===========================
Set up nlp object from config
INFO:spacy:Set up nlp object from config
Pipeline: ['textcat']
INFO:spacy:Pipeline: ['textcat']
Created vocabulary
INFO:spacy:Created vocabulary
Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Initialized pipeline components: ['textcat']
INFO:spacy:Initialized pipeline components: ['textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.50       33.55    0.34
  0     200        109.94       33.73    0.34
  0     400        104.41       66.76    0.67
  0     600        108.19       36.59    0.37
  0     800        111.10       51.53    0.52
  1    1000        107.34       56.68    0.57
  1    1200         88.20       55

In [ ]:
import spacy 
# load thebest model from training
nlp = spacy.load("output/model-best")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

/usr/local/lib/python3.7/dist-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  except PermissionError as e:


type : ‘quit’ to exit
Please enter example input: bad acting
{'positive': 5.555844836635515e-05, 'negative': 0.9999444484710693}
the sentiment is negative


KeyboardInterrupt: ignored

In [ ]:
!git clone https://github.com/sismetanin/rureviews

In [5]:
!pip install spacy==3.0.0
!pip install ml-datasets
! python3 -m spacy download ru_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.9 MB 1.3 MB/s 
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 8.2 MB 13.1 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=21aa1ee7ac34461211efec47a76fef4514b5a5c13421e3d2c4efcce6b12b196f
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [6]:
import pandas as pd
import spacy
from spacy.language import Language
import ru_core_news_sm

In [7]:
import spacy
import pandas as pd
from tqdm.auto import tqdm
from spacy.tokens import DocBin

In [8]:
nlp = spacy.load('ru_core_news_sm')

In [9]:
df = pd.read_csv("/content/rureviews/women-clothing-accessories.3-class.balanced.csv", sep = "\t")

In [10]:
df = df.sample(frac=1)

In [11]:
df.head

<bound method NDFrame.head of                                                   review sentiment
66980  почитала отзывы и заказала объем чуть больше, ...  positive
1788   Заказ сделала 08.11.17 .а получила 10.01.18 ((...  negative
27191  Категорически не рекомендую.. по крайней мере ...  negative
48353  Как на фото! Шла месяц до Перми! Сплошная синт...  neautral
77155  Доставка до Москвы 3 недели. Всё как в описани...  positive
...                                                  ...       ...
18809  Не подошло по размеру. В плечах и талии велико...  negative
13932  Девочки синтетика галимая, 100 рублей не стоит...  negative
31066                       Оказалась не как на картинке  neautral
78072  Юбка подошла по размеру :) Очень рада,спасибо,...  positive
82677                                        все отлично  positive

[90000 rows x 2 columns]>

In [12]:
df = df[df['sentiment']!= 'neautral']

In [13]:
y=df['sentiment']
X=df.drop(['sentiment'],axis=1)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [16]:
X_train = X_train['review'].to_list()

In [17]:
X_test = X_test['review'].to_list()

In [18]:
y_train = y_train.to_list()

In [19]:
data = list(zip(X_train, y_train))

In [20]:
valid_data = list(zip(X_test, y_test))

In [21]:
data[1]

('шикарный топ', 'positive')

In [22]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == 'negative':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [23]:
num_texts = 1000
# first we need to transform all the training data
train_docs = make_docs(data[:num_texts])

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [24]:
train_docs[0].cats



{'positive': 0, 'negative': 1}

In [25]:
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
%ls

 config.cfg   sample_data/  'Untitled Folder'/
 rureviews/   train.spacy    valid.spacy


In [26]:
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [27]:
! python -m spacy train config.cfg --output ./output

✔ Created output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
Set up nlp object from config
INFO:spacy:Set up nlp object from config
Pipeline: ['textcat']
INFO:spacy:Pipeline: ['textcat']
Created vocabulary
INFO:spacy:Created vocabulary
Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Initialized pipeline components: ['textcat']
INFO:spacy:Initialized pipeline components: ['textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.06       33.16    0.33
  0     200         19.21       83.82    0.84
  1     400          4.25       86.40    0.86
  2     600          0.82       88.40    0.88
  4     800          0.45       86.68    0.87
  5    1000          0.03       87.29    0.87

In [28]:
import spacy 
# load thebest model from training
nlp = spacy.load("output/model-best")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

type : ‘quit’ to exit
Please enter example input: хорошо
{'positive': 0.9973559379577637, 'negative': 0.002644131425768137}
the sentiment is positive


KeyboardInterrupt: ignored